Importing libraries 


In [1]:
from sqlalchemy import create_engine , inspect 
import pandas as pd

Defineing database , engine and showing how structure of database 

In [2]:
db_path = 'E:\\engenrring Hema\\Data Engineeing\\projects\\chinook analysis\\data source\\Chinook_Sqlite.sqlite'

engine = create_engine(f'sqlite:///{db_path}')

inspector = inspect(engine)

table_names = inspector.get_table_names()
print("Table Names : " , table_names)

for table in table_names : 
    print("Table Name : " , table , "===")

    columns = inspector.get_columns(table)

    col_names = [col['name'] for col in columns]
    print("Column Name : " , col_names)

    query = f'SELECT * FROM {table} LIMIT 5'
    df = pd.read_sql(query , engine)
    print(df)

Table Names :  ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Table Name :  Album ===
Column Name :  ['AlbumId', 'Title', 'ArtistId']
   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3
Table Name :  Artist ===
Column Name :  ['ArtistId', 'Name']
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains
Table Name :  Customer ===
Column Name :  ['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'Supp

Top 5 Customers by Total Spend (with breakdown by year)

In [3]:
query = '''
    WITH customer_spening 
    AS (
    SELECT  FirstName || ' ' ||LastName AS "Full Name",
            Country,
            strftime('%Y' , InvoiceDate) AS Year ,
            SUM(Total) AS "Total Spending" 
    FROM 'Invoice' AS  s
    INNER JOIN Customer AS c 
    ON s.CustomerId = C.CustomerId
    GROUP BY strftime('%Y' , InvoiceDate) , C.CustomerId
    )

    SELECT * 
    FROM (
    SELECT  * , 
            ROW_NUMBER() OVER(PARTITION BY Year ORDER BY "Total Spending" DESC) AS RN
    FROM customer_spening ) as partitioning_customers
    WHERE RN <= 5
'''

top5_spending_year = pd.read_sql(query,engine)
print(top5_spending_year)

             Full Name         Country  Year  Total Spending  RN
0        Leonie Köhler         Germany  2009           24.75   1
1            Tim Goyer             USA  2009           24.75   2
2   Dominique Lefebvre          France  2009           24.75   3
3          John Gordon             USA  2009           22.77   4
4      Alexandre Rocha          Brazil  2009           15.84   5
5      Ladislav Kovács         Hungary  2010           32.75   1
6        Astrid Gruber         Austria  2010           27.77   2
7    François Tremblay          Canada  2010           26.75   3
8        Frank Ralston             USA  2010           26.75   4
9           Luis Rojas           Chile  2010           17.91   5
10       Hugh O'Reilly         Ireland  2011           32.75   1
11        Daan Peeters         Belgium  2011           24.75   2
12        Robert Brown          Canada  2011           24.75   3
13       Enrique Muñoz           Spain  2011           22.77   4
14      Victor Stevens   

Most popular genre in each country

In [4]:
query = '''
    WITH genre_usage
    AS (
        SELECT  s.BillingCountry AS Country , 
            g.Name AS "Genre Name" , 
            SUM(Total) AS "Total Spending"
        FROM Invoice AS s
        INNER JOIN InvoiceLine sl
        ON s.InvoiceId = sl.InvoiceId
        INNER JOIN Track t
        ON sl.TrackId = t.TrackId
        INNER JOIN Genre g 
        ON t.GenreId = g.GenreId
        GROUP BY s.BillingCountry , g.Name 
    )
    SELECT  Country , 
            "Genre Name" , 
            "Total Spending" FROM 
    (
    SELECT  Country , 
            "Genre Name" , 
            "Total Spending", 
            ROW_NUMBER() OVER(PARTITION BY Country ORDER BY "Total Spending" DESC) AS RN
    FROM genre_usage
    ) AS ordered_genere_spending
    WHERE RN = 1
'''

mostGenre = pd.read_sql(query , engine)
print(mostGenre)

           Country Genre Name  Total Spending
0        Argentina      Latin           91.08
1        Australia       Rock          170.28
2          Austria       Rock          203.40
3          Belgium       Rock          209.88
4           Brazil       Rock          691.02
5           Canada       Rock          989.03
6            Chile      Latin           91.08
7   Czech Republic       Rock          226.02
8          Denmark       Rock          219.78
9          Finland       Rock          169.29
10          France       Rock          525.73
11         Germany       Rock          577.17
12         Hungary   TV Shows           87.44
13           India       Rock          242.55
14         Ireland       Rock          170.76
15           Italy       Rock          189.09
16     Netherlands       Rock          198.00
17          Norway       Rock          106.92
18          Poland       Rock          191.07
19        Portugal       Rock          289.14
20           Spain       Rock     

Artist Revenue Ranking & Customer Behavior

In [5]:
query = '''
    SELECT  art.Name AS "Artist Name" , 
            SUM(sl.UnitPrice * Quantity) AS "Total Revenue" 
    FROM InvoiceLine sl
    INNER JOIN Track t 
    ON sl.TrackId = t.TrackId
    INNER JOIN Album a
    ON t.AlbumId = a.AlbumId
    INNER JOIN Artist art
    ON art.ArtistId = a.ArtistId 
    GROUP BY art.Name 
    ORDER BY "Total Revenue" DESC
    '''

artist_revenue_ranking = pd.read_sql(query,engine)
print("Top 5 Artist Revenue Ranking")
print(artist_revenue_ranking.head(5))

### Customer Behavior

top_artist_revenue = artist_revenue_ranking.head(5)['Artist Name'].tolist()
artist_names_tuple = tuple(top_artist_revenue)
query2 = f'''
    WITH top_customer_spending_for_top_artist_revenue
    AS (
        SELECT  FirstName || " " || LastName AS fullName , 
            art.Name AS "Artist Name" ,
            SUM(Total) AS "Total Spending"
        FROM Customer c 
        INNER JOIN Invoice s 
        ON c.CustomerId = s.CustomerId 
        INNER JOIN InvoiceLine sl 
        ON s.InvoiceId = sl.InvoiceId
        INNER JOIN Track t
        ON sl.TrackId = t.TrackId
        INNER JOIN Album a 
        ON t.AlbumId = a.AlbumId
        INNER JOIN Artist art
        ON a.ArtistId = art.ArtistId
        WHERE art.Name IN {artist_names_tuple}
        GROUP BY c.CustomerId
    )
    SELECT fullName ,
            "Total Spending" , 
            "Artist Name"
    FROM (
        SELECT fullName ,
                "Total Spending" , 
                "Artist Name" ,
                ROW_NUMBER() OVER(PARTITION BY "Artist Name" ORDER BY "Total Spending" DESC) AS RN
        FROM top_customer_spending_for_top_artist_revenue ) AS newTable
    WHERE RN <= 3
    '''
customer_most_artistRevenue = pd.read_sql(query2,engine)
print("Customer Behavior for Top 5 Artist Revenue Ranking")
print(customer_most_artistRevenue)

Top 5 Artist Revenue Ranking
    Artist Name  Total Revenue
0   Iron Maiden         138.60
1            U2         105.93
2     Metallica          90.09
3  Led Zeppelin          86.13
4          Lost          81.59
Customer Behavior for Top 5 Artist Revenue Ranking
              fullName  Total Spending   Artist Name
0     Madalena Sampaio          263.34   Iron Maiden
1          Mark Taylor          254.43   Iron Maiden
2         Frank Harris          238.59   Iron Maiden
3    Jennifer Peterson           81.19  Led Zeppelin
4       João Fernandes           59.48  Led Zeppelin
5         Steve Murray           47.52  Led Zeppelin
6        Hugh O'Reilly          314.95          Lost
7        Astrid Gruber          308.59          Lost
8           Luis Rojas           89.37          Lost
9         Wyatt Girard           80.19     Metallica
10          Emma Jones           47.52     Metallica
11          Jack Smith           35.64     Metallica
12  Richard Cunningham          278.30       

Customer Lifetime Value by Cohort (First Year Purchase)

In [6]:
query = '''
WITH first_date_purchaseCustomer AS (
    SELECT CustomerId,
            MIN(InvoiceDate) AS first_date_purchase
        FROM Invoice
        GROUP BY CustomerId
    ),
    cohort_invoices AS (
        SELECT 
            strftime('%Y', fcus.first_date_purchase) AS cohort_year,
            strftime('%Y', s.InvoiceDate) AS invoice_year,
            s.CustomerId
        FROM Invoice s
        INNER JOIN first_date_purchaseCustomer fcus
            ON s.CustomerId = fcus.CustomerId
        GROUP BY cohort_year, invoice_year, s.CustomerId
    )
    SELECT 
        cohort_year,
        COUNT(DISTINCT CASE WHEN invoice_year = '2009' THEN CustomerId END) AS y2009,
        COUNT(DISTINCT CASE WHEN invoice_year = '2010' THEN CustomerId END) AS y2010,
        COUNT(DISTINCT CASE WHEN invoice_year = '2011' THEN CustomerId END) AS y2011,
        COUNT(DISTINCT CASE WHEN invoice_year = '2012' THEN CustomerId END) AS y2012,
        COUNT(DISTINCT CASE WHEN invoice_year = '2013' THEN CustomerId END) AS y2013
    FROM cohort_invoices
    GROUP BY cohort_year
    ORDER BY cohort_year;


    '''

analysis_cohort_retention = pd.read_sql(query , engine)
print(analysis_cohort_retention)

  cohort_year  y2009  y2010  y2011  y2012  y2013
0        2009     46     33     37     36     33
1        2010      0     13     10     11     13


Average Time Between Customer Purchases

In [7]:
query = '''
    SELECT CustomerId , AVG(JULIANDAY(Next_Order_date) - JULIANDAY(InvoiceDate)) AS GAB_In_Days
    FROM (
        SELECT CustomerId ,
                InvoiceDate , 
                LEAD(InvoiceDate , 1 , NULL)  OVER(PARTITION BY CustomerId ORDER BY CustomerId , InvoiceDate) AS Next_Order_date
        FROM Invoice s )
    GROUP BY CustomerId
    '''

gab_between_orders = pd.read_sql(query,engine)
print(gab_between_orders)

    CustomerId  GAB_In_Days
0            1   207.500000
1            2   214.833333
2            3   214.833333
3            4   289.166667
4            5   207.500000
5            6   264.333333
6            7   214.833333
7            8   289.166667
8            9   207.500000
9           10   264.333333
10          11   214.833333
11          12   266.000000
12          13   207.500000
13          14   264.333333
14          15   214.833333
15          16   266.000000
16          17   207.500000
17          18   207.500000
18          19   214.833333
19          20   214.833333
20          21   289.166667
21          22   207.500000
22          23   298.000000
23          24   214.833333
24          25   289.166667
25          26   207.500000
26          27   264.333333
27          28   214.833333
28          29   266.000000
29          30   207.500000
30          31   264.333333
31          32   214.833333
32          33   266.000000
33          34   207.500000
34          35   207

Sales by Media Type Over Time

In [8]:
query = '''
    SELECT mt.Name , strftime('%Y' , InvoiceDate) AS Year , SUM(sl.UnitPrice * sl.Quantity) AS "Total Revenue"
    FROM InvoiceLine sl
    INNER JOIN Track t 
    ON sl.TrackId = t.TrackId
    INNER JOIN MediaType mt
    ON mt.MediaTypeId = t.MediaTypeId
    INNER JOIN Invoice s
    ON s.InvoiceId = sl.InvoiceId
    GROUP BY mt.Name , strftime('%Y' , InvoiceDate)
    '''

most_mediatype_revenue = pd.read_sql(query , engine)
print(most_mediatype_revenue)

                           Name  Year  Total Revenue
0                AAC audio file  2010           0.99
1                AAC audio file  2011           0.99
2                AAC audio file  2012           0.99
3               MPEG audio file  2009         432.63
4               MPEG audio file  2010         373.23
5               MPEG audio file  2011         361.35
6               MPEG audio file  2012         371.25
7               MPEG audio file  2013         417.78
8      Protected AAC audio file  2009          16.83
9      Protected AAC audio file  2010          42.57
10     Protected AAC audio file  2011          42.57
11     Protected AAC audio file  2012          35.64
12     Protected AAC audio file  2013           6.93
13  Protected MPEG-4 video file  2010          61.69
14  Protected MPEG-4 video file  2011          63.68
15  Protected MPEG-4 video file  2012          69.65
16  Protected MPEG-4 video file  2013          25.87
17     Purchased AAC audio file  2010         

Playlist / Track Popularity Correlation

In [9]:
query = '''
    SELECT *
    FROM (
        SELECT  pl.Name AS "Playlist Name",
                COUNT(DISTINCT sl.TrackId) AS "Number of sold tracks" , 
                ROUND((CAST(COUNT(DISTINCT sl.TrackId) AS float) / COUNT(sl.TrackId)) * 100 , 1) AS "Percentage sold tracks" , 
                SUM(UnitPrice * Quantity) AS "Total Revenue"
        FROM Playlist pl
        INNER JOIN PlaylistTrack  plt
        ON pl.PlaylistId = plt.PlaylistId
        INNER JOIN InvoiceLine sl
        ON plt.TrackId = sl.TrackId
        GROUP BY pl.PlaylistId
        )
    GROUP BY "Playlist Name" 
    ORDER BY "Total Revenue" DESC
'''

analysis_playlist = pd.read_sql(query , engine)
print(analysis_playlist)

                Playlist Name  Number of sold tracks  Percentage sold tracks  \
0                       Music                   1881                    88.4   
1                  90’s Music                    848                    88.9   
2                    TV Shows                    103                    92.8   
3                   Classical                     36                    87.8   
4             Brazilian Music                     22                    81.5   
5         Heavy Metal Classic                     18                    81.8   
6   Classical 101 - Deep Cuts                     16                    84.2   
7  Classical 101 - Next Steps                     13                    86.7   
8                      Grunge                      7                   100.0   
9  Classical 101 - The Basics                      7                   100.0   

   Total Revenue  
0        2107.71  
1         944.46  
2         220.89  
3          40.59  
4          26.73  
5    